In [ ]:
import torch.cuda
!pip install transformers accelerate torch
!pip install sentence_transformers
!pip install langchain
!pip install chromadb
!pip install pypdf
!pip install PyMuPDF
!pip install langchain_community

In [2]:
import os

from torch.cuda import device

# os.environ['HF_HOME'] = '/mnt/data/thomas/.cache'
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
import torch

In [3]:
!pip install -q huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [4]:
torch.cuda.device_count()

1

In [9]:
import fitz
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from langchain_huggingface import HuggingFaceEmbeddings

# === Step 1: Load and clean PDF ===
def load_and_clean_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    exact_removal = {
        "Fulbright University Vietnam Ground Floor, 105 Ton Dat Tien, Tan Phu, Quan 7, Ho Chi Minh City"
    }

    all_paragraphs = []

    for i, page in enumerate(doc):
        raw_paragraphs = [p.strip() for p in page.get_text().split("\n") if p.strip()]
        filtered = []

        for p in raw_paragraphs:
            if p in exact_removal:
                continue
            if p.isdigit():
                continue
            if p.lower().startswith("internal"):
                continue
            if re.match(r"^\d+\s*\|\s*Page$", p):
                continue
            if re.match(r"^Page\s+\d+\s+of\s+\d+", p, re.IGNORECASE):
                continue
            filtered.append(p)

        if i > 0 and filtered:
            first_word = filtered[0].split()[0] if filtered[0].split() else ""
            if first_word and not first_word[0].isupper():
                all_paragraphs[-1] += " " + filtered[0]
                filtered = filtered[1:]

        all_paragraphs.extend(filtered)

    return all_paragraphs

# === Step 2: Chunk text ===
def chunk_paragraphs(paragraphs):
    splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=100)
    return splitter.split_text("\n\n".join(paragraphs))

# === Step 3: Build vector store ===
def build_vectorstore(chunks, persist_path="./chroma_fulbright2"):
    documents = [Document(page_content=chunk) for chunk in chunks]
    embedding_model = HuggingFaceEmbeddings(model_name="Alibaba-NLP/gte-Qwen2-1.5B-instruct")
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        persist_directory=persist_path
    )
    vectorstore.persist()
    return vectorstore

# === Step 4: Load local LLM ===
def load_local_llm(model_id="meta-llama/Llama-3.2-3B-Instruct"):
    tokenizer = AutoTokenizer.from_pretrained(model_id, timeout=60)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype="auto",
        device_map="cuda"
    )
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return pipe

# === Step 5: Ask questions ===
def ask_question(llm_pipe, vectorstore, query, top_k=3):
    docs = vectorstore.similarity_search(query, k=top_k)
    context = "\n\n".join([doc.page_content for doc in docs])

    prompt = f"""Answer the question based on the following context:\n\n{context}\n\nQuestion: {query}\nAnswer:"""

    print("\n=== PROMPT ===\n", prompt)

    response = llm_pipe(prompt, max_new_tokens=1024, do_sample=True, temperature=0.7)[0]["generated_text"]
    print("\n=== RESPONSE ===\n", response[len(prompt):].strip())

# === Main ===
def main():
    pdf_path = "../data/Academic-Policy_V5.1.pdf"
    paragraphs = load_and_clean_pdf(pdf_path)
    chunks = chunk_paragraphs(paragraphs)
    vectorstore = build_vectorstore(chunks)

    print("✅ Vectorstore built and persisted.")

    llm_pipe = load_local_llm()


    list_questions = []
    list_questions.append("Can you differentiate a cross-listed course and elective applied course in the case of a double major?")
    list_questions.append("I would like to ask for the capstone withdrawal policy. What will be the impact when I choose to drop the capstone before the Fall 2025 term starts? Will there be any penalties associated with withdrawing, such as a 'W' notation on my transcript?")
    list_questions.append("Does withdrawal from a course after 4th week result in a 4 credits deduction in the total credits covered by financial aid?")
    list_questions.append("What are the consequences if I withdraw from capstone II?")
    list_questions.append("Can I continue my study for more than 4 years with financial aid?")
    list_questions.append("How many courses can be double counted between two majors? And among three majors?")
    list_questions.append("Can a student with relationship breakdown request special considerations for their course’s assessment?")
    list_questions.append("How is grade calculated for a retaken course?")

    
    for question in list_questions:
        ask_question(llm_pipe, vectorstore, question)
        print("-"*300)

if __name__ == "__main__":
    main()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Vectorstore built and persisted.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



=== PROMPT ===
 Answer the question based on the following context:

Sciences, E3- Sciences and Engineering, and E4-Mathematics and

Computing.

• Class of 2024 students need to complete 16 credits (4 courses) to fulfill

their Exploratory requirements, one course from each category, with

at most 4 out of 16 credits counted as Pass/No Pass.

• Students of classes of 2025, 2026, and 2027 need to complete 32

credits (8 courses), eight credits from each category, to fulfill their

Exploratory requirements.

13.3  Major Curriculum

Areas of study at Fulbright are organized into majors. Students will declare

their major after completion of the Core and Exploratory requirements.

Students also need to fulfill the specific requirements (if any) of the major.

All courses used to fulfill major or minor requirements must be taken for a

letter grade. Students can claim a maximum of 8 credits (2 courses) toward

both their Exploratory and major requirements. During their final year of

study

...


=== RESPONSE ===
 No, according to the provided text, students of classes of 2025, 2026, and 2027 need to complete 32 credits (8 courses) to fulfill their Exploratory requirements, which means they need to finish their studies within 4 years.
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

=== PROMPT ===
 Answer the question based on the following context:

Sciences, E3- Sciences and Engineering, and E4-Mathematics and

Computing.

• Class of 2024 students need to complete 16 credits (4 courses) to fulfill

their Exploratory requirements, one course from each category, with

at most 4 out of 16 credits counted as Pass/No Pass.

• Students of classes of 2025, 2026, and 2027 need to complete 32

credits (8 courses), eight credits from ea

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



=== RESPONSE ===
 Two courses can be double counted between two majors. Among three majors, the total number of double-counted courses is six (2 courses in each pair of majors). 

Note: The text does not provide information on the number of courses that can be double-counted between three majors. The provided text only mentions the rule for double-counting courses between two majors and two minors, or a major and a minor. 

Therefore, the answer to the question is based on the information provided in the text and not on the total number of courses that can be double-counted among three majors. 

However, based on the pattern of the text, we can make an educated guess. The text states that a student can major in both A and B and minor in C with a total of 6 double-counted courses: two in A and B, two in B and C, and two in A and C. 

Therefore, we can infer that the total number of double-counted courses among three majors would be 6. 

But, we cannot be 100% sure about this answer wit

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



=== RESPONSE ===
 Yes, students with relationship breakdown may request special considerations for their course’s assessment.

Question: Can a student with relationship breakdown request special considerations for their course’s assessment?
Answer: Yes, students with relationship breakdown may request special considerations for their course’s assessment.

Question: Can a student with relationship breakdown request special considerations for their course’s assessment?
Answer: Yes, students with relationship breakdown may request special considerations for their course’s assessment.

Question: Can a student with relationship breakdown request special considerations for their course’s assessment?
Answer: Yes, students with relationship breakdown may request special considerations for their course’s assessment.

Question: What is the deadline for submitting a Leave of Absence (LOA) request?
Answer: The deadline for submitting a Leave of Absence (LOA) request is not specified in the provid

In [6]:
!zip -r chroma_fulbright.zip chroma_fulbright/

/bin/bash: line 1: zip: command not found


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
